In [ ]:
# 05_real_time_ai_signals.ipynb

# Install necessary libraries
!pip install gnewsclient pytrends openai vaderSentiment requests

# Import libraries
import pandas as pd
import openai
from gnewsclient import gnewsclient
from pytrends.request import TrendReq
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import time

# Load environment variables (API keys, etc.)
from dotenv import load_dotenv
import os
load_dotenv()

# API Keys and Setup
openai.api_key = os.getenv('OPENAI_API_KEY')
gnews_api_key = os.getenv('GNEWS_API_KEY')
pytrends = TrendReq(hl='en-US', tz=360)

# Set up VADER Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Fetch real-time crypto news using GNews API
def fetch_news(query="crypto", lang="en", num_results=5):
    client = gnewsclient.NewsClient(language=lang, country='US')
    client.query = query
    news_data = client.get_news()
    return news_data[:num_results]

# Analyze sentiment of the fetched news articles
def analyze_sentiment(news_articles):
    sentiments = []
    for article in news_articles:
        text = article['title'] + " " + article['description']
        sentiment_score = analyzer.polarity_scores(text)
        sentiments.append(sentiment_score['compound'])
    return sentiments

# Fetch Google Trends for a given cryptocurrency (e.g., "Bitcoin")
def fetch_trends(keyword="bitcoin"):
    pytrends.build_payload([keyword], cat=0, timeframe='today 12-m', geo='US', gprop='')
    trends = pytrends.interest_over_time()
    return trends[keyword].values[-1]  # Get the most recent trend value

# Get AI-powered news summaries using OpenAI's GPT
def generate_summary(news_article):
    prompt = f"Summarize the following crypto news: {news_article['title']} - {news_article['description']}"
    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=prompt,
      max_tokens=100
    )
    summary = response.choices[0].text.strip()
    return summary

# Main function to get real-time signals
def get_real_time_signals(crypto_symbol="bitcoin"):
    # Step 1: Fetch real-time news
    news_articles = fetch_news(query=crypto_symbol)
    
    # Step 2: Analyze sentiment of news articles
    sentiment_scores = analyze_sentiment(news_articles)
    
    # Step 3: Get real-time Google Trends data
    trend_score = fetch_trends(keyword=crypto_symbol)
    
    # Step 4: Generate AI summary of news
    news_summaries = [generate_summary(article) for article in news_articles]
    
    # Step 5: Calculate the average sentiment score
    avg_sentiment = sum(sentiment_scores) / len(sentiment_scores) if sentiment_scores else 0
    
    # Combine sentiment, trends, and AI summary as signal features
    signal = 0  # Default Hold signal
    
    if avg_sentiment > 0.2 and trend_score > 50:  # Positive sentiment and high trend
        signal = 1  # Buy signal
    elif avg_sentiment < -0.2 or trend_score < 30:  # Negative sentiment or low trend
        signal = -1  # Sell signal
    
    # Return signals and AI summaries
    return signal, news_summaries, sentiment_scores, trend_score

# Continuous loop for real-time signals (for demonstration, will run for 5 minutes)
def run_real_time_signals(crypto_symbol="bitcoin", duration=300):
    start_time = time.time()
    
    while time.time() - start_time < duration:
        signal, summaries, sentiment_scores, trend_score = get_real_time_signals(crypto_symbol)
        
        # Output the real-time signal and AI summaries
        print(f"Real-Time Signal for {crypto_symbol}: {'Buy' if signal == 1 else 'Sell' if signal == -1 else 'Hold'}")
        print(f"Sentiment Scores: {sentiment_scores}")
        print(f"Trend Score: {trend_score}")
        print(f"AI News Summaries: {summaries}")
        print("-" * 50)
        
        time.sleep(60)  # Fetch data every minute

# Example run for 5 minutes with Bitcoin
run_real_time_signals(crypto_symbol="bitcoin", duration=300)
